# Estimate Neural Manifold Curvature

### Set-up + Imports

In [ ]:
import setup

setup.main()
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

import os

import matplotlib.pyplot as plt
import numpy as np
import skdim

import neurometry.datasets.synthetic as synthetic
from neurometry.dimension.dimension import (
    plot_dimension_experiments,
    skdim_dimension_estimation,
)

os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs